In [ ]:
import pandas as pd
import numpy as np
import pylab as pl

from llckbdm.llckbdm import _cluster_line_lists,_transform_line_lists, llc_kbdm, iterative_llc_kbdm
from llckbdm.sampling import filter_samples
from llckbdm.min_rmse_kbdm import min_rmse_kbdm
from llckbdm.sig_gen import multi_fid, gen_t_freq_arrays

In [ ]:
columns = ['amplitude', 't2', 'frequency', 'phase']

df = pd.read_csv(
    'data/params_brain_sim_1_5T.csv',
    names=columns
)

In [ ]:
dwell = 5e-4
N = 2048
params = df.as_matrix()

t_array, freq_array = gen_t_freq_arrays(N, dwell)

data_raw = multi_fid(t_array, params)

In [ ]:
df = df.sort_values(['frequency'])
df['gamma'] = 1/df['t2']
df['phase_deg'] = df['phase'] * 180 / np.pi
df

In [ ]:
%matplotlib inline
noise = 0.03 * (np.random.randn(N) + 1j * np.random.randn(N))

data = data_raw + noise

#data[:4] += 1 * (np.random.randn(4) + 1j * np.random.randn(4))

pl.plot(data)
pl.show()

In [ ]:
data_fft = np.fft.fftshift(np.fft.fft(data)) / np.sqrt(N)
data_fft_raw = np.fft.fftshift(np.fft.fft(data_raw)) / np.sqrt(N)

pl.plot(freq_array, data_fft.real)
pl.plot(freq_array, data_fft_raw.real)
pl.plot(freq_array, data_fft.real - data_fft_raw.real )

pl.xlim(-100, 750)
pl.show()

In [ ]:
m_range = range(200, 500, 10)
result = iterative_llc_kbdm(
    data=data, 
    dwell=dwell, 
    m_range=m_range, 
    p=4, 
    gep_solver='svd', 
    l=30, 
    q=0, 
    max_iterations=20, silhouette_threshold=0.92
)

In [ ]:
df_final_est = pd.DataFrame(data=result.line_list, columns=columns)
df_final_est['silhouete'] = np.concatenate(result.silhouettes)
df_final_est['gamma'] = 1 / df_final_est['t2']
df_final_est['phase_deg'] = df_final_est['phase'] * 180 / np.pi
df_final_est.sort_values(by='frequency', inplace=True, ascending=False)
df_final_est = df_final_est[(df_final_est['silhouete'] > 0.6) & (df_final_est['amplitude'] > 0.00)]
df_final_est = df_final_est[(df_final_est['frequency'] > 300) & (df_final_est['frequency'] < 450)]
#df_final_est.head()
df_final_est

In [ ]:
len(df_final_est)

In [ ]:
%matplotlib notebook
%matplotlib notebook
%matplotlib notebook

params_res_est = df_final_est.as_matrix()[:,0:4]

data_res_est = multi_fid(t_array, params_res_est)
data_fft_res_est = np.fft.fftshift(np.fft.fft(data_res_est)) / np.sqrt(N)

pl.plot(freq_array, data_fft.real)

# raw
pl.plot(freq_array, data_fft_raw.real, '--', 'green')

# estimatated
pl.plot(freq_array, data_fft_res_est.real, 'black')

# true residue
pl.plot(freq_array, data_fft_res_est.real - data_fft_raw.real, 'red')

pl.xlim(-100, 750)

pl.show()